# **과제1 : 챗봇만들기**

## 0.미션

* 예비 에이블러들을 위한 QA 챗봇 모델 만들기1
    * Vector DB 준비하기
    * Retriever, LLM를 연결하기

## **1.환경준비**

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00


In [ ]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.1 MB/s eta 0:00:00


In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 4

In [ ]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.8 MB/s eta 0:00:00


### (1) 라이브러리 Import

In [ ]:
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime
import chromadb

import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [ ]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key)

* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [ ]:
os.environ['OPENAI_API_KEY'] = "YOUR_API_KEY"
openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Vector DB 만들기**

* 데이터 로딩

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/KTaivle/7차미니프로젝트/미니프로젝트 7차_실습자료/과제파일/aivleschool_qa.csv', encoding='utf-8')
data.head()

,구분,QA
0,모집/선발,최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제...
1,모집/선발,35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하는 교...
2,모집/선발,미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함...
3,모집/선발,"직장인도 지원할 수 있나요?\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시..."
4,모집/선발,아르바이트를 하고 있는데 지원할 수 있나요?\n고용보험에 가입이 되어 있는 경우 1...


* 벡터 데이터베이스
    * Embedding 모델 : text-embedding-ada-002
    * DB 경로 : ./db



In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

database = Chroma(persist_directory="./data",
                    embedding_function = embeddings
)

* 데이터 입력
    * DF to Vector DB

In [ ]:
temp_data = list(data["QA"])

In [ ]:
ind = database.add_texts(temp_data)
ind

['0acfc912-2334-41b8-9b87-1b8e6c87f168',
 'a0666f0b-4ce2-4c2c-af88-07b812bcaa46',
 '358e1afc-4281-475e-8467-d57f325c4da0',
 '9d6992f9-df62-49a9-a926-4580ad9bf070',
 '9e8c434c-959e-4a2f-a86c-9d122d614c95',
 '0910ad77-2f43-4a5b-b9fb-b5b89059212f',
 '0993bf49-8e70-4fda-b10f-e516ff4ad97e',
 '96603b9d-f377-4cc9-82ff-21347cbdd51b',
 '0f7cf7b0-048b-4742-98e2-020ae5008477',
 '4b786df9-9c82-4e92-97aa-9dc0dd579474']

* 입력된 데이터 조회

In [ ]:
database.get(ind)

{'ids': ['0910ad77-2f43-4a5b-b9fb-b5b89059212f',
  '0993bf49-8e70-4fda-b10f-e516ff4ad97e',
  '0acfc912-2334-41b8-9b87-1b8e6c87f168',
  '0f7cf7b0-048b-4742-98e2-020ae5008477',
  '358e1afc-4281-475e-8467-d57f325c4da0',
  '4b786df9-9c82-4e92-97aa-9dc0dd579474',
  '96603b9d-f377-4cc9-82ff-21347cbdd51b',
  '9d6992f9-df62-49a9-a926-4580ad9bf070',
  '9e8c434c-959e-4a2f-a86c-9d122d614c95',
  'a0666f0b-4ce2-4c2c-af88-07b812bcaa46'],
 'embeddings': None,
 'metadatas': [None, None, None, None, None, None, None, None, None, None],
 'documents': ['사업자도 지원할 수 있나요?\n사업자는 자영업자로 분류됩니다. 자영업자의 경우 관할 고용센터에서 국민내일배움카드 발급 여부를 검토하며, 카드가 발급되어 교육을 수강하여도 훈련장려금은 지급되지 않습니다. 다만, 고용보험을 임의가입한 자영업자의 경우 훈련장려금이 지급됩니다.',
  '프리랜서도 지원할 수 있나요?\n고용보험에 가입하지 않은 경우 미취업자로 보지만, 프리랜서 계약의 경우 개인별 상황이 다를 수 있어서 관할 고용센터에 문의하여 확인하시기 바랍니다.\n※ 관할 고용센터 :\n수도권: 성남고용센터\n충남/충북: 대전고용센터\n대구/경북: 대구고용센터\n전남/전북: 광주고용센터\n부산/경남: 부산고용센터\n- 고용노동부 대표번호 1350, 거주지 근처 지도검색, HRD-Net 메인 하단 지역별 기관검색에서 확인 가능',
  '최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제

## **3.RAG+LLM모델**

* 모델 : RetrievalQA
    * LLM 모델 : gpt-3.5-turbo
    * retriever : 벡터DB
        * 유사도 높은 문서 3개 가져오도록 설정

* database를 retriever로 선언하기

In [ ]:
chat = ChatOpenAI(model="gpt-3.5-turbo")
retriever = database.as_retriever()

* 모델 선언

In [ ]:
qa = RetrievalQA.from_llm(llm=chat,  retriever=retriever,  return_source_documents=True, )

query = input()
result = qa(query)

에이블스쿨의 취업률을 알려줘


* 모델 확인

In [ ]:
print(result["result"])

죄송하지만, 에이블스쿨의 취업률에 대한 정보는 현재 제가 알고 있는 바가 없습니다. 실시간으로 업데이트되는 정보이므로, 공식 웹사이트나 해당 기관에 직접 문의하여 최신 정보를 확인하시는 것이 좋을 것 같습니다.


In [35]:
k = 3
retriever = database.as_retriever(search_kwargs={"k": k})

qa = RetrievalQA.from_llm(llm=chat,  retriever=retriever,  return_source_documents=True, )

result = qa(query)
result

{'query': '에이블스쿨의 취업률을 알려줘',
 'result': '죄송합니다. 현재 에이블스쿨의 취업률에 대한 정보는 제가 알고 있는 바가 없습니다. 에이블스쿨의 취업률에 대한 정확한 정보는 해당 학교나 기관의 공식 웹사이트나 취업 보고서를 확인하시는 것이 좋습니다.',
 'source_documents': [Document(page_content='직장인도 지원할 수 있나요?\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가능합니다. 향후 취업여부를 확인할 예정입니다.'),
  Document(page_content='미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함이 있는 근로인 경우,\n2) 고용보험에 미가입한 경우,\n3) 고용보험에 가입되어 있더라도 15시간/주 미만 근로인 경우\n단, 어떠한 경우에도 교육을 풀타임(09:00~18:00)으로 들을 수 있어야 교육 참여가 가능합니다.'),
  Document(page_content='최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 하는 교육입니다. 전공에 관계 없이 명시된 지원 자격에 부합한다면 모두 지원 가능합니다. 다만, AI개발자 Track은 기본적인 코딩역량이 필요합니다.')]}